# 프롬프트 엔지니어링 with ChatGPT

## Prompt 설정


1. **`temperature`**
    - **작동 방식**:  
      확률 분포를 **평탄화하거나 집중**시킵니다.
      - 낮은 값 (`0.1` 등): 높은 확률의 토큰만 선택하게 하여 응답이 더 **결정적**(예측 가능)입니다.
      - 높은 값 (`1.5` 등): 확률 분포를 평탄화하여 저확률 토큰도 선택할 가능성이 있어 응답이 더 **창의적**이고 다양합니다.
    
    - **적용 범위**:
      - 문장이 단조롭거나 응답이 너무 예상 가능할 때 다양성을 부여.
      - 확률 분포를 조정하는 가장 기본적인 매개변수.
    
    - **비유**:
      - 음식을 만들 때 **조미료의 양을 조정**하는 것처럼, 다양성을 직접적으로 조절.

2. **`top-p`** (또는 **nucleus sampling**)
    - **작동 방식**:  
      확률 분포에서 누적 확률이 특정 값(`p`) 이하가 되는 상위 토큰들만 고려합니다.  
      - 예: `top-p=0.9`라면, 전체 확률의 **90%를 차지하는 상위 토큰**들만 선택 후보로 유지.
      - 확률 분포의 **꼬리 부분**(즉, 낮은 확률 토큰)을 잘라내어 선택 가능성을 제한합니다.
    
    - **적용 범위**:
      - 확률이 높은 후보군에 집중하면서도 너무 단조로운 응답을 방지.
      - 매우 드문(낮은 확률) 선택지를 배제.
    
    - **비유**:
      - 음식에서 **상위 재료 90%만 골라서 요리**하는 것처럼, 유의미한 선택만 남김.
3. **`최대 길이(Max Length)`**
    - 생성되는 토큰 수의 최대 길이를 제한한다.
    - 길이를 제한하여 응답 품질과 비용을 제어한다.
4. **`중지 시퀀스(Stop Sequences)`**
    - 특정 문자열에서 응답 생성을 중단한다.
    - 응답의 구조와 길이를 제어할 수 있다.
5. **`빈도 패널티(Frequency Penalty)`**
    - 특정 단어가 자주 반복되지 않도록 페널티를 적용한다.
    - 빈도 패널티가 높을수록 단어가 다시 나타날 가능성이 낮아짐.
    - 더 많이 나타나는 토큰에 더 높은 패널티를 부여함으로써 모델의 응답에서 단어의 반복을 줄임
6. **`존재 패널티(Presence Penalty)`**
    - 특정 단어가 한 번이라도 등장하면 페널티를 적용한다.
    - 빈도 패널티와 달리, 반복된 토큰에 대한 패널티를 적용하지만, 모든 반복된 토큰에 대해 패널티가 동일하다.
        - 두 번 나타나는 토큰과 열 번 나타나는 토큰이 같은 패널티를 받는다.
    - 모델이 응답에서 구절을 너무 자주 반복하는 것을 방지
    - 모델에게 다양하거나 창의적인 텍스트를 생성하도록 하고 싶다면 높은 존재 패널티를 사용


**temperature vs. top-p**

| **특징**           | **temperature**                        | **top-p**                            |
|---------------------|----------------------------------------|--------------------------------------|
| **작동 방식**       | 확률 분포를 평탄화하거나 집중          | 누적 확률 기준으로 후보 제한         |
| **결과 다양성**     | 전체 분포에서 다양성을 조정            | 높은 확률 토큰만 사용해 제어          |
| **조정 방식**       | 분포의 "모양"을 변경                  | 분포의 "범위"를 제한                 |
| **활용 사례**       | 모델의 창의성이나 결정성 직접 제어     | 비현실적 선택 방지                   |

**`temperature`와 `top-p`는 함께 결합해 사용할 수 있다**:
- `temperature=0.7, top-p=0.9`
- `temperature`로 창의성을 설정하고, `top-p`로 지나치게 낮은 확률의 토큰을 배제하여 모델이 **실용적이면서도 창의적**인 결과를 생성하게 만들수 있다.
  

### Chat Completion의 구성

**프롬프트 구성요소**

- **지시사항 `User Message`**: 수행하고자 하는 구체적인 작업에 대한 지시.
- **컨텍스트 `System Instruction`**: 페르소나 등의 배경정보, 외부 정보나 추가 세부사항을 제공하여 모델 응답을 보완.
- **입력 데이터**: 응답을 유도하기 위한 데이터나 질문.
- **출력 지시자**: 응답의 형식과 유형을 명시.


**Role**

| Role         | 설명                                                                                     | 사용 시점                                                                                          |
|--------------|------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
| `system`     | 대화의 초기 설정과 규칙을 정의                                                           | 대화가 시작될 때 모델의 성격, 행동, 스타일을 설정하는 메시지에서 사용                               |
| `user`       | 사용자가 모델에 요청이나 질문을 전달                                                    | 대화 중에 사용자가 요청하거나 질문할 때 사용                                                      |
| `assistant`  | 모델이 사용자에게 응답하거나 제안                                                      | 사용자의 질문에 답변하거나 함수 호출을 제안할 때 사용                                              |
| `function`   | 함수 호출을 처리하거나 함수의 반환값을 전달                                             | 모델이 함수 호출을 제안하거나 함수가 반환값을 전달할 때 사용                                       |


In [1]:
!pip install openai

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Chat Completion

chatbot과 대화형 응답을 할 수 있는 Endpoint

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages="",
    response_format=,
    temperature=1.0,         # 대답 창의성
    max_tokens=2048,         # 응답 최대 토큰 수
    top_p=1.0,               # 사용할 상위 누적 확률
    frequency_penalty=0.0,   # 토큰 사용 빈도수에 대한 불이익
    presence_penalty=0.0     # 토큰 재사용에 대한 불이익
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable